# 6장. 파이썬과 MySQL로 구축하는 주소 데이터베이스 

- 작성자: 박하람
- 작성일자: 2024.02.22

## 6.2 프로젝트 환경 구축하기 

### 파이썬 노트북으로 MySQL 연결하기

In [1]:
!pip3 install pymysql

In [16]:
import pymysql

def init_connection():
    connection = pymysql.connect(
        host="localhost",
        port=3306,
        user="root",
        password="root",
        cursorclass=pymysql.cursors.DictCursor,
        charset="utf8",
    )
    return connection

sql = "SHOW DATABASES;"
conn = init_connection()

with conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        for data in cur:
            print(data)

{'Database': 'information_schema'}
{'Database': 'mysql'}
{'Database': 'performance_schema'}
{'Database': 'sys'}


## 6.3 데이터베이스 생성하기

### 데이터베이스 생성하기

In [17]:
conn = init_connection()
sql = "CREATE DATABASE address;"

with conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        conn.commit()

In [18]:
conn = init_connection()

sql = "SHOW DATABASES;"

with conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        for data in cur:
            print(data)

{'Database': 'address'}
{'Database': 'information_schema'}
{'Database': 'mysql'}
{'Database': 'performance_schema'}
{'Database': 'sys'}


## 6.4 테이블 생성하기

### 테이블 생성하기

In [25]:
def init_db_connection():
    connection = pymysql.connect(
        host="localhost",
        port=3306,
        user="root",
        password="root",
        database="address",
        cursorclass=pymysql.cursors.DictCursor,
        charset="utf8",
    )
    return connection

def query_get(sql):
    connection = init_db_connection()
    with connection:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            return cursor.fetchall()

def query_update(sql):
    connection = init_db_connection()
    with connection:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            connection.commit()
            return True

In [23]:
sql = '''
    CREATE TABLE `rnaddrkor` (
        `도로명주소관리번호` varchar(26) NOT NULL,
        `법정동코드` varchar(10),
        `시도명` varchar(40),
        `시군구명` varchar(40),
        `읍면동명` varchar(40),
        `리명` varchar(40),
        `산여부` varchar(1),
        `번지` varchar(4),
        `호` varchar(4),
        `도로명코드` varchar(12) NOT NULL,
        `도로명` varchar(80),
        `지하여부` varchar(1) NOT NULL,
        `건물본번` int(5) ZEROFILL NOT NULL,
        `건물부번` int(5) ZEROFILL NOT NULL,
        `행정동코드` varchar(60),
        `행정동명` varchar(60),
        `기초구역번호(우편번호)` varchar(5),
        `이전도로명주소` varchar(400),
        `효력발생일` varchar(8),
        `공동주택구분` varchar(1),
        `이동사유코드` varchar(2),
        `건축물대장건물명` varchar(400),
        `시군구용건물명` varchar(400),
        `비고` varchar(200),
        PRIMARY KEY (`도로명주소관리번호`, `도로명코드`, `지하여부`, `건물본번`, `건물부번`)
    );
'''

query_update(sql)

True

In [28]:
sql = '''
    CREATE TABLE `rnaddrkor_jibun` (
        `도로명주소관리번호` varchar(26) NOT NULL,
        `법정동코드` varchar(10) NOT NULL,
        `시도명` varchar(40),
        `시군구명` varchar(40),
        `법정읍면동명` varchar(40),
        `법정리명` varchar(40),
        `산여부` varchar(1) NOT NULL,
        `지번본번(번지)` int(4) ZEROFILL NOT NULL,
        `지번부번(호)` int(4) ZEROFILL NOT NULL,
        `도로명코드` varchar(12),
        `지하여부` varchar(1),
        `건물본번` int(5),
        `건물부번` int(5),
        `이동사유코드` varchar(2),
        PRIMARY KEY (`도로명주소관리번호`, `법정동코드`, `산여부`, `지번본번(번지)`, `지번부번(호)`)
    );
'''

query_update(sql)

True

In [29]:
query_get("SHOW TABLES;")

[{'Tables_in_address': 'rnaddrkor'}, {'Tables_in_address': 'rnaddrkor_jibun'}]

In [27]:
query_get("DESC rnaddrkor;")

[{'Field': '도로명주소관리번호',
  'Type': 'varchar(26)',
  'Null': 'NO',
  'Key': 'PRI',
  'Default': None,
  'Extra': ''},
 {'Field': '법정동코드',
  'Type': 'varchar(10)',
  'Null': 'YES',
  'Key': '',
  'Default': None,
  'Extra': ''},
 {'Field': '시도명',
  'Type': 'varchar(40)',
  'Null': 'YES',
  'Key': '',
  'Default': None,
  'Extra': ''},
 {'Field': '시군구명',
  'Type': 'varchar(40)',
  'Null': 'YES',
  'Key': '',
  'Default': None,
  'Extra': ''},
 {'Field': '읍면동명',
  'Type': 'varchar(40)',
  'Null': 'YES',
  'Key': '',
  'Default': None,
  'Extra': ''},
 {'Field': '리명',
  'Type': 'varchar(40)',
  'Null': 'YES',
  'Key': '',
  'Default': None,
  'Extra': ''},
 {'Field': '산여부',
  'Type': 'varchar(1)',
  'Null': 'YES',
  'Key': '',
  'Default': None,
  'Extra': ''},
 {'Field': '번지',
  'Type': 'varchar(4)',
  'Null': 'YES',
  'Key': '',
  'Default': None,
  'Extra': ''},
 {'Field': '호',
  'Type': 'varchar(4)',
  'Null': 'YES',
  'Key': '',
  'Default': None,
  'Extra': ''},
 {'Field': '도로명코드',
  '

## 6.5 데이터 삽입하기

## 6.6 SQL로 데이터 분석하기